In [19]:
import pandas as pd
import random
import datetime


In [20]:
# number of records in the ola tablr
num_rows = 50

# number of dates to generate data for
num_dates = 7

# number of jobs having more that one run for each date
num_dup_jobs = 2

# define aggregation keys for the reporting - that will be specific columns of the ola table
key1 = 'Application'
key2 = 'Category'
key3 = 'Description'

In [21]:
def generate_ola_records(num_rows):
    data = []

    for i in range(num_rows):
        application = random.choice(['APP_1', 'APP_2'])
        category = random.choice(['CAT_1', 'CAT_2', 'CAT_3'])
        description = random.choice(['DESC_1', 'DESC_2'])
        job_name = f'JOB_{i+1}'
        cutoff = datetime.time(random.randint(0, 6), 0)
        active = random.choices([True, False], weights=[90, 10], k=1)[0]
        source = random.choice(['SRC_1', 'SRC_2'])
        daymatch = random.choices([True, False], weights=[90, 10], k=1)[0]
        nextday = random.choices([True, False], weights=[90, 10], k=1)[0]
        saturday = random.choices([True, False], weights=[10, 90], k=1)[0]

        # Append the data to the list
        data.append([application, category, description, job_name, cutoff, active, source, daymatch, nextday, saturday])

    # Create the DataFrame
    df = pd.DataFrame(data, columns=['Application', 'Category', 'Description', 'Job Name', 'Cutoff', 'Active', 'Source', 'Daymatch', 'Nextday', 'Saturday'])

    # Return the DataFrame
    return df

# Call the function and assign the returned DataFrame to ola_records
ola_records = generate_ola_records(num_rows)

In [22]:

def get_dates(num_dates):
    # Get today's date
    today = pd.Timestamp.today().normalize()

    # Initialize a list to store the dates
    dates = []

    # Start from 2 days ago and keep going back until we have enough dates
    i = 2
    while len(dates) < num_dates:
        date = today - pd.DateOffset(days=i)
        if date.dayofweek < 5:  # If the date is not on a weekend
            dates.append(date)
        i += 1

    return dates


def get_start_time(date):
    # Create a start time
    return datetime.datetime.combine(date, datetime.time(random.randint(20, 23), random.randint(0, 59))) if random.random() < 0.5 else datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 1), random.randint(0, 59)))


def get_end_time(date):
    # Create an end time
    return datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 7), random.randint(0, 59)))

In [23]:
def generate_data(ola_records, num_dates, num_rows,num_dup_jobs):
    # Create a list of unique job names
    job_names = ola_records['Job Name'].unique().tolist()

    data = []
    dates = get_dates(num_dates)
    for i in range(num_dates):
        date = dates[i]
        available_job_names = job_names.copy()  # Create a copy of job names for each date
        for j in range(num_rows + num_dup_jobs):
            start_time = get_start_time(date)
            end_time = get_end_time(date)

            # If all job names are used for this date, refresh the available job names
            if not available_job_names:
                available_job_names = job_names.copy()

            # Select a job name from the available job names and remove it from the list
            job_name = random.choice(available_job_names)
            available_job_names.remove(job_name)

            # Append the data to the list
            data.append([date, job_name,start_time, end_time])

    # Create the DataFrame
    df = pd.DataFrame(data, columns=['Date',  'Job Name','Start Time', 'End Time'])

    # Return the DataFrame
    return df
# Call the function with the ola_records DataFrame
data_df = generate_data(ola_records,num_dates, num_rows, num_dup_jobs)

In [24]:

data_df

,Date,Job Name,Start Time,End Time
0,2023-12-15,JOB_28,2023-12-15 20:18:00,2023-12-16 03:36:00
1,2023-12-15,JOB_32,2023-12-15 21:28:00,2023-12-16 00:52:00
2,2023-12-15,JOB_5,2023-12-16 01:19:00,2023-12-16 04:52:00
3,2023-12-15,JOB_44,2023-12-16 00:43:00,2023-12-16 02:32:00
4,2023-12-15,JOB_14,2023-12-16 00:54:00,2023-12-16 06:58:00
...,...,...,...,...
359,2023-12-07,JOB_20,2023-12-08 00:49:00,2023-12-08 03:55:00
360,2023-12-07,JOB_43,2023-12-08 01:07:00,2023-12-08 04:49:00
361,2023-12-07,JOB_36,2023-12-08 01:19:00,2023-12-08 03:47:00
362,2023-12-07,JOB_6,2023-12-08 01:57:00,2023-12-08 02:19:00


In [25]:
data_df.to_csv('data.csv', index=False)

In [26]:
# Identify duplicates based on 'Date' and 'Job Name'
duplicates = data_df[data_df.duplicated(['Date', 'Job Name'], keep=False)]

duplicates.to_csv('duplicates.csv', index=False)

# Print the duplicates
print(duplicates)

          Date Job Name          Start Time            End Time
13  2023-12-15    JOB_1 2023-12-15 22:25:00 2023-12-16 05:07:00
44  2023-12-15   JOB_43 2023-12-15 23:42:00 2023-12-16 01:08:00
50  2023-12-15    JOB_1 2023-12-16 00:22:00 2023-12-16 06:11:00
51  2023-12-15   JOB_43 2023-12-15 20:54:00 2023-12-16 04:11:00
57  2023-12-14   JOB_11 2023-12-14 21:25:00 2023-12-15 05:46:00
78  2023-12-14   JOB_31 2023-12-15 01:37:00 2023-12-15 01:13:00
102 2023-12-14   JOB_31 2023-12-14 22:38:00 2023-12-15 03:42:00
103 2023-12-14   JOB_11 2023-12-15 00:45:00 2023-12-15 06:26:00
131 2023-12-13   JOB_38 2023-12-14 00:50:00 2023-12-14 07:28:00
143 2023-12-13   JOB_29 2023-12-14 01:54:00 2023-12-14 02:34:00
154 2023-12-13   JOB_38 2023-12-13 20:13:00 2023-12-14 07:13:00
155 2023-12-13   JOB_29 2023-12-13 23:16:00 2023-12-14 05:26:00
159 2023-12-12   JOB_18 2023-12-12 23:08:00 2023-12-13 01:01:00
198 2023-12-12    JOB_7 2023-12-13 01:23:00 2023-12-13 02:14:00
206 2023-12-12   JOB_18 2023-12-13 01:46

In [27]:
merged_df = data_df.merge(ola_records, on='Job Name', how='inner')

merged_df

,Date,Job Name,Start Time,End Time,Application,Category,Description,Cutoff,Active,Source,Daymatch,Nextday,Saturday
0,2023-12-15,JOB_28,2023-12-15 20:18:00,2023-12-16 03:36:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
1,2023-12-14,JOB_28,2023-12-15 01:06:00,2023-12-15 03:12:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
2,2023-12-13,JOB_28,2023-12-14 01:52:00,2023-12-14 06:22:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
3,2023-12-12,JOB_28,2023-12-12 20:39:00,2023-12-13 07:40:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
4,2023-12-11,JOB_28,2023-12-11 20:23:00,2023-12-12 07:40:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2023-12-13,JOB_27,2023-12-14 00:54:00,2023-12-14 00:41:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False
360,2023-12-12,JOB_27,2023-12-12 20:48:00,2023-12-13 01:04:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False
361,2023-12-11,JOB_27,2023-12-12 01:47:00,2023-12-12 04:42:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False
362,2023-12-08,JOB_27,2023-12-08 23:28:00,2023-12-09 04:31:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False


In [28]:
# Ask the user if they want to remove records before a specified date
remove_records = input("Do you want to remove records before a specified date? (yes/no): ")

if remove_records.lower() == "yes":
    # Ask the user to enter the specified date
    specified_date = input("Enter the specified date (YYYY-MM-DD): ")

    # Convert the specified date to a pandas Timestamp object
    specified_date = pd.Timestamp(specified_date)

    # Filter the merged_df table to remove records before the specified date
    merged_df = merged_df[merged_df['Date'] >= specified_date]


In [29]:
merged_df

,Date,Job Name,Start Time,End Time,Application,Category,Description,Cutoff,Active,Source,Daymatch,Nextday,Saturday
0,2023-12-15,JOB_28,2023-12-15 20:18:00,2023-12-16 03:36:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
1,2023-12-14,JOB_28,2023-12-15 01:06:00,2023-12-15 03:12:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
2,2023-12-13,JOB_28,2023-12-14 01:52:00,2023-12-14 06:22:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
3,2023-12-12,JOB_28,2023-12-12 20:39:00,2023-12-13 07:40:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
4,2023-12-11,JOB_28,2023-12-11 20:23:00,2023-12-12 07:40:00,APP_2,CAT_3,DESC_2,03:00:00,True,SRC_1,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2023-12-13,JOB_27,2023-12-14 00:54:00,2023-12-14 00:41:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False
360,2023-12-12,JOB_27,2023-12-12 20:48:00,2023-12-13 01:04:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False
361,2023-12-11,JOB_27,2023-12-12 01:47:00,2023-12-12 04:42:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False
362,2023-12-08,JOB_27,2023-12-08 23:28:00,2023-12-09 04:31:00,APP_2,CAT_2,DESC_2,06:00:00,True,SRC_2,True,True,False


In [30]:
# Create a new column 'Cutoff Datetime' in the merged_df DataFrame
merged_df['Cutoff Datetime'] = merged_df.apply(lambda row: row['Date'] + pd.DateOffset(days=1) if row['Nextday'] else row['Date'], axis=1)

# Adjust the cutoff datetime based on the day of the week and the 'Saturday' column
merged_df['Cutoff Datetime'] = merged_df.apply(lambda row: row['Cutoff Datetime'] + pd.DateOffset(days=2) if row['Date'].dayofweek == 4 and not row['Saturday'] else row['Cutoff Datetime'], axis=1)

# Convert 'Cutoff' column from datetime.time to number of hours past midnight
merged_df['Cutoff'] = merged_df['Cutoff'].apply(lambda t: t.hour + t.minute/60 + t.second/3600)

# Add the number of hours in the 'Cutoff' column to the 'Cutoff Datetime' column
merged_df['Cutoff Datetime'] = merged_df['Cutoff Datetime'] + pd.to_timedelta(merged_df['Cutoff'], unit='h')

# Print the updated DataFrame
merged_df


,Date,Job Name,Start Time,End Time,Application,Category,Description,Cutoff,Active,Source,Daymatch,Nextday,Saturday,Cutoff Datetime
0,2023-12-15,JOB_28,2023-12-15 20:18:00,2023-12-16 03:36:00,APP_2,CAT_3,DESC_2,3.0,True,SRC_1,True,True,False,2023-12-18 03:00:00
1,2023-12-14,JOB_28,2023-12-15 01:06:00,2023-12-15 03:12:00,APP_2,CAT_3,DESC_2,3.0,True,SRC_1,True,True,False,2023-12-15 03:00:00
2,2023-12-13,JOB_28,2023-12-14 01:52:00,2023-12-14 06:22:00,APP_2,CAT_3,DESC_2,3.0,True,SRC_1,True,True,False,2023-12-14 03:00:00
3,2023-12-12,JOB_28,2023-12-12 20:39:00,2023-12-13 07:40:00,APP_2,CAT_3,DESC_2,3.0,True,SRC_1,True,True,False,2023-12-13 03:00:00
4,2023-12-11,JOB_28,2023-12-11 20:23:00,2023-12-12 07:40:00,APP_2,CAT_3,DESC_2,3.0,True,SRC_1,True,True,False,2023-12-12 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2023-12-13,JOB_27,2023-12-14 00:54:00,2023-12-14 00:41:00,APP_2,CAT_2,DESC_2,6.0,True,SRC_2,True,True,False,2023-12-14 06:00:00
360,2023-12-12,JOB_27,2023-12-12 20:48:00,2023-12-13 01:04:00,APP_2,CAT_2,DESC_2,6.0,True,SRC_2,True,True,False,2023-12-13 06:00:00
361,2023-12-11,JOB_27,2023-12-12 01:47:00,2023-12-12 04:42:00,APP_2,CAT_2,DESC_2,6.0,True,SRC_2,True,True,False,2023-12-12 06:00:00
362,2023-12-08,JOB_27,2023-12-08 23:28:00,2023-12-09 04:31:00,APP_2,CAT_2,DESC_2,6.0,True,SRC_2,True,True,False,2023-12-11 06:00:00


In [31]:
# Calculate the buffer column
merged_df['Buffer'] = (merged_df['Cutoff Datetime'] - merged_df['End Time']).dt.total_seconds() / 3600

# Sort the table by ascending dates, application, category, description, and job names
merged_df.sort_values(by=['Date', 'Application', 'Category', 'Description', 'Job Name'], inplace=True)

# Drop duplicates and keep the row with the smallest buffer
merged_df.drop_duplicates(subset=['Date', 'Application', 'Category', 'Description', 'Job Name'], keep='first', inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Print the updated DataFrame
merged_df

# save to csv
merged_df.to_csv('merged.csv', index=False)


In [32]:

pivot_table = merged_df.pivot_table(index=[key1, key2, key3], columns='Date', values='Buffer', aggfunc='min').reset_index()

pivot_table


Date,Application,Category,Description,2023-12-07 00:00:00,2023-12-08 00:00:00,2023-12-11 00:00:00,2023-12-12 00:00:00,2023-12-13 00:00:00,2023-12-14 00:00:00,2023-12-15 00:00:00
0,APP_1,CAT_1,DESC_1,-2.366667,3.683333,-1.550000,-4.950000,-0.016667,1.233333,1.016667
1,APP_1,CAT_1,DESC_2,-6.300000,44.316667,-5.016667,-4.116667,-6.200000,-3.133333,42.333333
2,APP_1,CAT_2,DESC_1,-3.433333,-2.700000,-2.133333,-5.400000,-4.950000,-1.116667,1.116667
3,APP_1,CAT_2,DESC_2,-6.616667,44.066667,-5.266667,-2.066667,-4.900000,-6.983333,43.883333
4,APP_1,CAT_3,DESC_1,-27.983333,19.083333,-29.750000,-27.533333,-30.200000,-27.800000,20.133333
5,APP_1,CAT_3,DESC_2,-29.083333,24.450000,-24.500000,-26.783333,-28.433333,-27.033333,20.883333
6,APP_2,CAT_1,DESC_1,-5.966667,1.000000,-2.516667,-3.550000,-4.883333,-2.650000,-5.833333
7,APP_2,CAT_1,DESC_2,-23.633333,18.433333,-27.766667,-26.433333,-28.416667,-24.866667,18.033333
8,APP_2,CAT_2,DESC_1,-2.750000,45.066667,-2.266667,-3.383333,-5.666667,-1.966667,45.800000
9,APP_2,CAT_2,DESC_2,-4.300000,44.316667,-3.150000,0.016667,-1.816667,-2.150000,45.600000


In [55]:
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import os

# Create a new workbook
workbook = openpyxl.Workbook()

# Get the active sheet
sheet = workbook.active

# Write the pivot table to the sheet
for r in dataframe_to_rows(pivot_table, index=False, header=True):
    sheet.append(r)

# Apply conditional formatting to the buffer cells
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=4, max_col=sheet.max_column):
    for cell in row:
        if cell.value is not None:
            if cell.value > 0:
                cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
            elif cell.value < -1:
                cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
            else:
                cell.fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")

# Adjust column width
for column in sheet.columns:
    if column[0].column_letter in [key1, key2, key3]:
        sheet.column_dimensions[column[0].column_letter].width = 16
    else:
        sheet.column_dimensions[column[0].column_letter].width = 10

# Round the values to the first decimal place
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=4, max_col=sheet.max_column):
    for cell in row:
        if cell.value is not None:
            cell.value = round(cell.value, 1)

# Format the dates on the first row
for cell in sheet[1][3:]:
    cell.number_format = 'dd-mmm-yy'

# Rename the sheet
sheet.title = 'Worst 3 levels'

# Save the workbook as an Excel file
workbook.save('pivot_table.xlsx')

# Open the workbook in Excel
os.system('open pivot_table.xlsx -a "Microsoft Excel"')


0